# Content-based Filtering

### Loading Libraries

In [245]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import numpy as np
warnings.filterwarnings('ignore')

### Loading Datasets

In [246]:
movies = pd.read_csv('../data/ml-25m/movies.csv')
movies_with_genres = pd.read_csv('../data/movies_with_genres.csv')
movies_with_year = pd.read_csv('../data/movies_with_year.csv', sep=';')
print(movies.columns)

Index(['movieId', 'title', 'genres'], dtype='object')


### Titel bereinigen

In [247]:
# Extrahiere das Erscheinungsjahr aus dem Titel und füge es als neue Spalte hinzu
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

# Entferne das Erscheinungsjahr aus dem Titel
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

movies without year

In [248]:
# Filme ohne Erscheinungsjahr ausgeben
movies_without_year = movies[movies['year'].isna()]
print("the number of movies without year is: ", movies['year'].isna().sum())

# Füge die fehlenden Jahre aus der neuen CSV-Datei ein, basierend auf den Titeln
movies['year'] = movies.apply(
    lambda row: movies_with_year[movies_with_year['title'] == row['title']]['year'].values[0]
    if pd.isnull(row['year']) and row['title'] in movies_with_year ['title'].values
    else row['year'],
    axis=1
)

# Filme ohne Erscheinungsjahr ausgeben
movies_without_year1 = movies[movies['year'].isna()]
print("the number of movies without year is: ", movies['year'].isna().sum())

# Entferne Filme ohne Erscheinungsjahr
movies = movies.dropna(subset=['year'])
movies.isnull().sum()

print(movies.head(5))

the number of movies without year is:  410
the number of movies without year is:  83
   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


### Genres aufbereiten


In [249]:
# Filme ohne Genre ausgeben
print("Anzahl der Filme mit '(no genres listed)':", movies[movies['genres'] == '(no genres listed)'].shape[0])

# Zusammenführen der DataFrames basierend auf den Titeln und Jahren
movies_updated = pd.merge(movies, movies_with_genres[['title', 'year', 'genres']], on=['title', 'year'], how='left', suffixes=('', '_new'))

# Aktualisieren der Genres im movies DataFrame nur für die übereinstimmenden Einträge
movies_updated['genres'] = movies_updated['genres_new'].combine_first(movies_updated['genres'])

# Entferne die temporäre Spalte
movies_updated = movies_updated.drop(columns=['genres_new'])
movies = movies_updated

# Filme ohne Genre ausgeben
print("Anzahl der Filme mit '(no genres listed)':", movies[movies['genres'] == '(no genres listed)'].shape[0])

# Entferne "(no genres listed)" aus der Genre-Liste
movies['genres'] = movies['genres'].replace('(no genres listed)', '')

# Trenne die Genres in separate Listen
genre_list = movies['genres'].str.split('|')

# Finde alle einzigartigen Genres
all_genres = set(genre for sublist in genre_list for genre in sublist if genre)

# Erstelle für jedes Genre eine Spalte und fülle sie mit binären Werten
for genre in all_genres:
    movies[genre] = movies['genres'].apply(lambda x: int(genre in x.split('|')))

# Entferne die ursprüngliche 'genres' Spalte
movies = movies.drop(columns=['genres'])

print(movies.head(2))

Anzahl der Filme mit '(no genres listed)': 5002
Anzahl der Filme mit '(no genres listed)': 1257
   movieId      title  year  Horror  Thriller  History  Romance  Adult  \
0        1  Toy Story  1995       0         0        0        0      0   
1        2    Jumanji  1995       0         0        0        0      0   

   Documentary  Musical  ...  Fantasy  War  Game-Show  Biography  Action  \
0            0        0  ...        1    0          0          0       0   
1            0        0  ...        1    0          0          0       0   

   Sport  Comedy  Animation  Family  IMAX  
0      0       1          1       0     0  
1      0       0          0       0     0  

[2 rows x 32 columns]


### Nach Null-Werten suchen

In [250]:
movies.info()
movies.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62340 entries, 0 to 62339
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      62340 non-null  int64 
 1   title        62340 non-null  object
 2   year         62340 non-null  object
 3   Horror       62340 non-null  int64 
 4   Thriller     62340 non-null  int64 
 5   History      62340 non-null  int64 
 6   Romance      62340 non-null  int64 
 7   Adult        62340 non-null  int64 
 8   Documentary  62340 non-null  int64 
 9   Musical      62340 non-null  int64 
 10  Short        62340 non-null  int64 
 11  Adventure    62340 non-null  int64 
 12  Film-Noir    62340 non-null  int64 
 13  Western      62340 non-null  int64 
 14  Music        62340 non-null  int64 
 15  Reality-TV   62340 non-null  int64 
 16  Mystery      62340 non-null  int64 
 17  Drama        62340 non-null  int64 
 18  News         62340 non-null  int64 
 19  Sci-Fi       62340 non-nu

movieId        0
title          0
year           0
Horror         0
Thriller       0
History        0
Romance        0
Adult          0
Documentary    0
Musical        0
Short          0
Adventure      0
Film-Noir      0
Western        0
Music          0
Reality-TV     0
Mystery        0
Drama          0
News           0
Sci-Fi         0
Crime          0
Children       0
Fantasy        0
War            0
Game-Show      0
Biography      0
Action         0
Sport          0
Comedy         0
Animation      0
Family         0
IMAX           0
dtype: int64

### Map user input to get movieId

In [251]:
# Normalisiere die Titel-Spalte: leerzeichen entfernen, kleinbuchstaben, führende Artikel entfernen und ans Ende verschieben

def normalize_title(title):
    articles = ['the', 'a', 'an']
    words = title.strip().split()
    if words[-1].strip(",").lower() in articles:
        return title.strip().lower()
    if words[0].lower() in articles:
        return ', '.join(words[1:]) + ', ' + words[0].capitalize()
    return title.lower()

def alternate_title_format(title):
    articles = ['the', 'a', 'an']
    words = title.strip().split()
    if words[0].lower() in articles:
        return ', '.join(words[1:]) + ', ' + words[0].capitalize()
    if words[-1].strip(",").lower() in articles:
        return words[-1].capitalize() + ' ' + ' '.join(words[:-1]).replace(',', '')
    return title.lower()

# Funktion zur Zuordnung der IDs
def map_movie_ids(movies_df, user_ratings_input, cinema_movies_input):
    # Normalisieren der Titelspalte des DataFrames
    movies_df['normalized_title'] = movies_df['title'].apply(normalize_title)
    movies_df['alternate_title'] = movies_df['title'].apply(alternate_title_format)

    def get_movie_id(title, year):
        normalized_title = normalize_title(title)
        alternate_title = alternate_title_format(title)
        
        filtered_movies = movies_df[((movies_df['normalized_title'] == normalized_title) | (movies_df['alternate_title'] == alternate_title)) & (movies_df['year'] == str(year))]
        if not filtered_movies.empty:
            return filtered_movies.iloc[0]['movieId']
        else:
            print(f"{title} ({year}) not found")
            return None

    # IDs zu den Filmen im user_ratings_input Array hinzufügen
    mapped_user_ratings = []
    for rating in user_ratings_input:
        movie_id = get_movie_id(rating['title'], rating['year'])
        mapped_user_ratings.append({**rating, 'movieId': movie_id})
    
    # IDs zu den Filmen im cinema_movies_input Array hinzufügen
    mapped_cinema_movies = []
    for movie in cinema_movies_input:
        movie_id = get_movie_id(movie['title'], movie['year'])
        mapped_cinema_movies.append({**movie, 'movieId': movie_id})
    
    return mapped_user_ratings, mapped_cinema_movies

### Feature Set: Genre

In [252]:
# Funktion zur Extraktion von Filmmerkmalen
def get_movie_features(movies, movie_id):
    filtered_movie = movies[movies['movieId'] == movie_id]
    if not filtered_movie.empty:
        # Wähle nur die numerischen Spalten aus, die für die Berechnung der Ähnlichkeiten verwendet werden sollen, außer 'movieId'
        numeric_features = filtered_movie.drop(columns=['movieId']).select_dtypes(include=[np.number])
        return numeric_features.iloc[0]
    else:
        print(f"Movie with ID {movie_id} not found")
        return None

# Funktion zur Empfehlung von Filmen
def recommend_movies(user_ratings_input, cinema_movies_input):
    
    # Schritt 1: Extrahieren der Merkmale der vom Nutzer bewerteten Filme
    user_movie_features = []
    has_positive_ratings = any('rating' in movie and movie['rating'] >= 2.5 for movie in user_ratings_input)

    for movie in user_ratings_input:
        if has_positive_ratings and movie.get('rating', 0) >= 2.5:
            features = get_movie_features(movies, movie['movieId'])
            if features is not None:
                user_movie_features.append(features.values)
        elif not has_positive_ratings and movie.get('rating', 0) < 2.5:
            features = get_movie_features(movies, movie['movieId'])
            if features is not None:
                user_movie_features.append(features.values)
    
    # Schritt 2: Feature-Vektorisierung
    if not user_movie_features:
        raise ValueError("Keine positiv bewerteten Filme vorhanden.")
    
    user_profile = np.mean(user_movie_features, axis=0)
    
    # Schritt 3: Ähnlichkeitsberechnung
    kino_movie_features = []
    for kino_movie in cinema_movies_input:
        features = get_movie_features(movies, kino_movie['movieId'])
        if features is not None:
            kino_movie_features.append(features.values)
    
    if not kino_movie_features:
        raise ValueError("Keine Kino-Filme mit passenden Features gefunden.")
    
    similarities = cosine_similarity([user_profile], kino_movie_features)[0]
    similarities = similarities * 100
    
    # Schritt 4: Sortierung und Ausgabe
    # Falls der Nutzer positive Bewertungen abgegeben hat, sortiere nach absteigender Ähnlichkeit
    # Andernfalls sortiere nach aufsteigender Ähnlichkeit
    cinema_movies_with_similarity = []
    for i, kino_movie in enumerate(cinema_movies_input):
        kino_movie_with_similarity = {
            "externalId": kino_movie['externalId'],
            "movieTitle": kino_movie['title'],
            "year": kino_movie['year'],
            "score": similarities[i]
        }
        cinema_movies_with_similarity.append(kino_movie_with_similarity)
    
    if has_positive_ratings:
        sorted_cinema_movies = sorted(cinema_movies_with_similarity, key=lambda x: x['score'], reverse=True)
    else:
        sorted_cinema_movies = sorted(cinema_movies_with_similarity, key=lambda x: x['score'])
    
    return sorted_cinema_movies

### Test

In [253]:
# Beispiel-Daten
user_rated_movies = [
    {"externalId": "1", "title": "Emil and the Detectives", "year": 1964, "rating": 4.5},
    {"externalId": "2", "title": "Pride and Prejudice", "year": 2003, "rating": 1.5},
    {"externalId": "3", "title": "La La Land", "year": 2016, "rating": 2.0}
]
cinema_movies = [
    {"externalId": "4", "title": "Black Panther", "year": 2017},
    {"externalId": "5", "title": "Avengers: Infinity War - Part I", "year": 2018},
    {"externalId": "6", "title": "A Star Is Born", "year": 2018},
    {"externalId": "7", "title": "Bohemian Rhapsody", "year": 2018},
    {"externalId": "8", "title": "Deadpool 2", "year": 2018},
    {"externalId": "9", "title": "Spider-Man: Into the Spider-Verse", "year": 2018},
    {"externalId": "10", "title": "Crazy Rich Asians", "year": 2018},
    {"externalId": "11", "title": "Mission: Impossible - Fallout", "year": 2018},
    {"externalId": "12", "title": "Solo: A Star Wars Story", "year": 2018},
    {"externalId": "13", "title": "Aquaman", "year": 2018},
    {"externalId": "14", "title": "The Meg", "year": 2018},
    {"externalId": "15", "title": "Ready Player One", "year": 2018},
    {"externalId": "16", "title": "Jurassic World: Fallen Kingdom", "year": 2018}
]

# Map IDs
mapped_user_ratings, mapped_movies  = map_movie_ids(movies, user_rated_movies, cinema_movies)
# print("Mapped Movies:", mapped_movies)
# print("Mapped User Ratings:", mapped_user_ratings)
# Funktion aufrufen
recommended_cinema_movies = recommend_movies(mapped_user_ratings, mapped_movies)
for movie in recommended_cinema_movies:
    print(f"External ID: {movie['externalId']}, Title: {movie['movieTitle']}, Year: {movie['year']}, Score: {movie['score']:.2f}")

External ID: 4, Title: Black Panther, Year: 2017, Score: 57.74
External ID: 5, Title: Avengers: Infinity War - Part I, Year: 2018, Score: 57.74
External ID: 8, Title: Deadpool 2, Year: 2018, Score: 57.74
External ID: 11, Title: Mission: Impossible - Fallout, Year: 2018, Score: 57.74
External ID: 9, Title: Spider-Man: Into the Spider-Verse, Year: 2018, Score: 50.00
External ID: 10, Title: Crazy Rich Asians, Year: 2018, Score: 50.00
External ID: 12, Title: Solo: A Star Wars Story, Year: 2018, Score: 50.00
External ID: 16, Title: Jurassic World: Fallen Kingdom, Year: 2018, Score: 44.72
External ID: 13, Title: Aquaman, Year: 2018, Score: 28.87
External ID: 15, Title: Ready Player One, Year: 2018, Score: 28.87
External ID: 14, Title: The Meg, Year: 2018, Score: 25.00
External ID: 6, Title: A Star Is Born, Year: 2018, Score: 0.00
External ID: 7, Title: Bohemian Rhapsody, Year: 2018, Score: 0.00


### Evaluate Content-Based Filtering

In [254]:
from sklearn.model_selection import train_test_split

# Laden der Ratings-Daten
ratings = pd.read_csv('../data/ml-25m/ratings.csv')

# Splitten der Daten in Trainings- und Testdatensets
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

# Überprüfen der Splits
print(f"Trainingsdaten: {train_ratings.shape}")
print(f"Testdaten: {test_ratings.shape}")

FileNotFoundError: [Errno 2] No such file or directory: 'ratings.csv'